In [1]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
from numpy.random import randint
import matplotlib

# Necessary Functions

In [2]:
def encode_message(bits, bases):
    """
    Encodes 'bits' in 'bases'.
    n = length of the sequence
    """
    message = []
    for i in range(len(bits)):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0:
            if bits[i] == 0:
                pass
            else:
                qc.x(0)
        else:
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def decode_message(message, bases):
    """
    Decodes 'message' in the given set of 'bases'.
    """
    simulator = Aer.get_backend('qasm_simulator')
    decoded_message = []
    for i in range(len(message)):
        qc = message[i]
        if bases[i] == 0: # Z basis
            pass
        else: # X basis
            qc.h(0)
        qc.measure(0,0)
        result = execute(qc, backend=simulator, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        decoded_message.append(measured_bit)
    return decoded_message

def remove_garbage(a_basis, b_basis, bits):
    """
    Compares Alice's and Bob's bases and returns
    only those bits that have been decoded correctly.
    """
    good_bits = []
    for i in range(len(a_basis)):
        if a_basis[i] == b_basis[i]:
            good_bits.append(bits[i])
    return good_bits

def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [7]:
n = 100 # Number of qubits

# Alice generates her random set of bits
alice_bits = randint(2, size=n)

# Alice encodes her qubits, sends it to Bob
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)

# Eve butting in
eve_bases = randint(2, size=n)
intercepted_message = decode_message(message, eve_bases)

# Bob decodes message with his own set of bases
bob_bases = randint(2, size=n)
bob_results = decode_message(message, bob_bases)

# They compare messages and obtain a key
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

# Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

In [8]:
if bob_sample != alice_sample:
    print("Eve's interference was detected.")
else:
    print("Eve went undetected!")

Eve's interference was detected.
